In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# define loss function
class My_loss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, x, y):
        return torch.mean(torch.pow((x - y), 2))
    
def f(x):
    return (1+np.pi**2)*np.cos(np.pi*x)

In [10]:
# define model
class My_model(nn.Module):
    def __init__(self,hidden_layer1th):
        super().__init__()
        self.fc1 = nn.Linear(1,hidden_layer1th)
        self.fc2 = nn.Linear(hidden_layer1th,1)
    def forward(self,x):
        out = self.fc1(x)#xsize x hidden_layer1th
        out = F.relu(out)
        out = self.fc2(out)#xsize x 1
        out = F.relu(out)
        return out

In [34]:
# Hyper-parameters
N = 500
# width of 1th hidden layer
hidden_layer1th = 50
num_epochs = 60
learning_rate = 0.001
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# architecture
nodes = torch.zeros(N, 1,dtype=torch.float32)
nodes_np = np.linspace(0,1,N)
for i in range(N):
    nodes[i] = nodes_np[i];
targets = f(nodes)

criterion = My_loss()
model = My_model(hidden_layer1th).to(DEVICE)
optimizer = optim.Adam(model.parameters())

In [35]:
h = 1e-3

# define train
def train(model, device, optimizer, epoch):
    model.train()
    # uN-uN'' on each nodes
    outputs = model(nodes)# uN
    outputs -= (model(nodes-h)-2*model(nodes)+model(nodes+h))/h**2
    # targets = f(nodes)
    loss = criterion(outputs, targets)
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 5 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))


for epoch in range(num_epochs):
    train(model,DEVICE,optimizer,epoch)

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [500, 1]], which is output 0 of ReluBackward0, is at version 1; expected version 0 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

In [ ]:
# plot
outputs = model(nodes)
plt.plot(nodes,abs(outputs-targets))
plt.show()